In [40]:
import pandas as pd
import numpy as np
from tkinter import ttk
from tkinter import filedialog
from tkinter import *
from tkinter.ttk import *
import os

In [41]:
# Inputs and Parameters

# File Path
filePath = "C:/Users/marym/OneDrive/Documents/PropellerCalc/propellers/PER3_20x10E.dat"
file = "PER3_20x10E"
fileName = 'C:/Users/marym/OneDrive/Documents/PropellerCalc/propellers/' + file + '.dat'

# Constants
R = 8.3132                       # Gas Constant (J/K-mol)
M = 0.02896                      # Molar Mass of Earth's Air (kg)
g = 9.806                        # Force of Gravity (N)
P0 = 101325                      # Pressure at Sea Level (Pa) - Double Check Units with this value

# Starting Point
propDia = 20.0 * 0.0254          # Prop Diameter (in)
alt = 1289 * 0.3048              # Altitude w/ coversion (m)
alt = 0.0
T = 15 + 273.15 - 6.5 * alt / 1000   # Ambient Temp (K)
Pressure = P0 * np.exp(-M * g *  alt/ (R * T))    # Pressure at Alt (Pa)
Density = Pressure * M / (R * T) # Air Density ()
Density = 1.225

Throttle = 100.0                 # Throttle value (0 to 100)

# ESC
eCurrMax = 120.0                 # Max current of esc (A      -- Double Check these
eRes = 0.004                     # esc Resistance (milliOhms)      values!!

# Battery Cell -- From LiPos
cells = 6.0                      # Num cells
batVolt = 3.7                    # LiPo per-cell Voltage (V)
batRes = 0.0028                  # bat Resistance (milliOhms)

# Motor
kv = 295.0                       # RPM per Volt 
nLCurr = 1.7                     # No Load Current (A)
nLVolt = 8.4                     # No Load Voltage (V)
mRes = 0.015                     # Motor Resistnace (milliOhms)
mWatt = 2000.0                   # Max Watts of Motor (A)


print('propDiameter: '+str(propDia))
print('altitude: '+str(alt))
print('density: '+str(Density))
print('eCurrMax: '+str(eCurrMax))
print('eRes: '+str(eRes))      
print('battRes: '+str(batRes))           
print('battVolt: '+str(batVolt))
print('cells: '+str(cells))
print('kV: '+str(kv))
print('nLCurr: '+str(nLCurr))
#print('nLVolt: '+str(nLVolt))
print('mRes: '+str(mRes))
print('mWatt: '+str(mWatt))
print('Throttle: '+str(Throttle))

propDiameter: 0.508
altitude: 0.0
density: 1.225
eCurrMax: 120.0
eRes: 0.004
battRes: 0.0028
battVolt: 3.7
cells: 6.0
kV: 295.0
nLCurr: 1.7
mRes: 0.015
mWatt: 2000.0
Throttle: 100.0


In [42]:
""" Pulling Data """

# Set Column Index for Importing Data
header_list = [str(x) for x in range(13)]

# Read file
propData = pd.read_csv(fileName, sep = '\s+', names = header_list)
propData = propData.to_numpy()
df = pd.DataFrame(propData)
df.to_csv(index = False)

# Find All Rows w/ RPM Data
searchString = "RPM"
mask = np.column_stack([df[col].str.contains(r'' + searchString, na = False)
                        for col in df])
RPMdata = pd.DataFrame.dropna(df.loc[mask.any(axis = 1)], axis = 1)

In [43]:
"""
Create New Table to bin RPM Values into Groups
Remove Characters from PROP RPM
Change to Numeric Values for Analysis
"""

# First Find Column w/ Numbers
RPMs = RPMdata.apply(pd.to_numeric, errors = 'ignore')

# Second Remove All non-int64 Columns
RPMs = RPMs.select_dtypes(include = ['int64'])

# Third Get First RPM Value Index
a = RPMs.columns.values
indexinRPM = RPMs[a[0]].index[0]

# Delete Unnecessary Top Rows
RPMdf = df.iloc[indexinRPM:]

# Get Max RPM and Index
maxRPM = RPMs.max()
indexMaxRPM = RPMs[a[0]].idxmax()

# Get Index of Each RPM Value to Bin Data
RPMindex = RPMs.index

# Clear Columns w/ Mostly NA
RPMdf = pd.DataFrame.dropna(RPMdf, axis = 'columns', thresh = 10)

# Add Index of Last Row of Database ot RPM DF to Bin
lastIndex = RPMdf.iloc[[-1]].index.values[0]
# Get Index of RPMs ot Group by RPM
RPMindex = np.append(RPMindex, lastIndex)
RPMlabel = RPMs[a[0]].to_list()

# Label Columns - Assumes consistantcy in value-to-columns, V in first column
Units = RPMdf[RPMdf[0].str.match('V')]

# First Row of "V"
unitsRow = Units[0].index[0]

# Takes First Row, Sets as Column Headers
RPMdf.columns = RPMdf.loc[unitsRow]

# Removes All Non-Numeric Rows
RPMdf = RPMdf.apply(lambda x: pd.to_numeric(x, errors = 'coerce')).dropna()

# Group Each RPM
RPMdf = RPMdf.groupby(pd.cut(RPMdf.index, RPMindex,
                             right = False, labels = RPMlabel))

# Printing Out Grouped DataFrame
for name, group in RPMdf:
    print(name)
    print(group)

1000
9      V     J      Pe      Ct      Cp    PWR  Torque  Thrust
11   0.0  0.00  0.0000  0.0885  0.0330  0.008   0.535   0.451
12   0.5  0.02  0.0633  0.0874  0.0332  0.009   0.539   0.446
13   0.9  0.05  0.1239  0.0862  0.0335  0.009   0.543   0.439
14   1.4  0.07  0.1818  0.0847  0.0336  0.009   0.546   0.432
15   1.8  0.10  0.2368  0.0831  0.0338  0.009   0.549   0.424
16   2.3  0.12  0.2889  0.0814  0.0339  0.009   0.550   0.415
17   2.7  0.14  0.3380  0.0795  0.0340  0.009   0.551   0.405
18   3.2  0.17  0.3841  0.0775  0.0340  0.009   0.551   0.395
19   3.6  0.19  0.4270  0.0753  0.0339  0.009   0.551   0.384
20   4.1  0.22  0.4666  0.0729  0.0339  0.009   0.549   0.372
21   4.6  0.24  0.5030  0.0704  0.0337  0.009   0.547   0.359
22   5.0  0.26  0.5361  0.0678  0.0335  0.009   0.543   0.345
23   5.5  0.29  0.5660  0.0649  0.0331  0.009   0.537   0.331
24   5.9  0.31  0.5926  0.0618  0.0326  0.008   0.530   0.315
25   6.4  0.34  0.6158  0.0586  0.0320  0.008   0.520   0.298
26 

In [44]:
""" Notes & Shortened Terms:
        Curr = Current
        Res = Resistance
        Bat = Battery
        Volt = Voltage
        v = Voltage (when put before term)
        p = Power (when put before term)
        Amps = Amperage
        df = Data Frame
        J = Velocity / (PRs * Diameter (m)) - Prop Advance Ratio
"""

' Notes & Shortened Terms:\n        Curr = Current\n        Res = Resistance\n        Bat = Battery\n        Volt = Voltage\n        v = Voltage (when put before term)\n        p = Power (when put before term)\n        Amps = Amperage\n        df = Data Frame\n        J = Velocity / (PRs * Diameter (m)) - Prop Advance Ratio\n'

In [45]:
""" First Calculations """

RPMgroups = np.array(list(RPMdf.groups))

# allAmps array w/ Expected Amp Draw at Max Static Thrust at each velocity
allAmps = []

for groups in RPMgroups:
    RPMgroup = RPMdf.get_group(groups)
    PWR = RPMgroup['PWR'].iloc[0] * 745.7  # Converting HP to Watts
    allAmps.append(PWR / (cells * batVolt))
    
velocityArray = [0]
velocityStep = 1.524
    
def RPMfinder(motorAmpGuess, batAmpGuess, Output, velocity):
    
    # Main Calc for Iteration of pMotorOut
    def pMotorOutCalc(motorAmp, batAmp):
    
        vBat = cells * batVolt - cells * batRes * batAmp         # Total Bat Volts w/ Volt Loss
        vTerm = (vBat - eRes * batAmp) * Throttle / 100          # Volt Drop from esc/Throttle Val
        vDropMotor = motorAmp * mRes                             # Volt Drop from Motor
        pMotorIn = vTerm * batAmp                                # Power Motor Uses during Opt
        vEMF = vTerm - vDropMotor                                # Voltage Motor after losses
        pMotorOut = vEMF * (motorAmp - nLCurr)                   # Power Motor Generates (Voltage of Motor * (Amps of Motor - No Load Motor Amps))
        pMotorIn = vTerm * (motorAmp - nLCurr)                   # Power Motor Receives

        return pMotorIn, pMotorOut, vTerm, motorAmp, vBat, batAmp, vEMF
    
    pMotorIn, pMotorOut, vTerm, motorAmp, vBat, batAmp, vEMF = pMotorOutCalc(motorAmpGuess, batAmpGuess)
    
    RPM = vEMF * kv                # Revolutions/Min of Motor
    RPS = RPM / 60                 # Revolutions/s of Motor
    
    
    if (batAmp > batVolt/batRes):
        raise Exception("Batteries can't supply this much current: " + str(batAmp))
    if (batAmpGuess > eCurrMax):
        raise Exception("ESC will be overloaded, chose larger ESC: " + str(Amps))
    if (RPM < 0):
        raise Exception("RPM is negative, something is wrong: " + str(RPM))
    
    
    # Find Closest Data to RPM Values, above and below
    RPMgroups = np.array(list(RPMdf.groups))
    
    if (RPM > RPMgroups.max()):
        raise Exception("RPM max limit has been reached")
    if (RPM < RPMgroups.min()):
        raise Exception('RPM minimum limit reached')
    
    bottomRPM = RPMgroups[RPMgroups < RPM].max()
    topRPM = RPMgroups[RPMgroups > RPM].min()
    
    # Get Bottom/Top RPM Groups
    bottomRPMdata = RPMdf.get_group(bottomRPM).reset_index()
    topRPMdata = RPMdf.get_group(topRPM).reset_index()
    
    velocityMPH = velocity * 2.23694      # Conversion to MPH
    
    # Finding Relevant Indices
    bottomRPMClosestVel = abs(bottomRPMdata['V'] - velocityMPH).idxmin()
    topRPMClosestVel = abs(topRPMdata['V'] - velocityMPH).idxmin()
    
    # Finding Ct
    bottomCt = bottomRPMdata['Ct'].iloc[bottomRPMClosestVel]  # Coefficeient of Thrust
    topCt = topRPMdata['Ct'].iloc[topRPMClosestVel]
    
    # Finding Cp
    bottomCp = bottomRPMdata['Cp'].iloc[bottomRPMClosestVel]
    topCp = topRPMdata['Cp'].iloc[topRPMClosestVel]
    
    # Finding Torque
    bottomTorque = bottomRPMdata['Torque'].iloc[bottomRPMClosestVel]
    topTorque = topRPMdata['Torque'].iloc[topRPMClosestVel]
    
    # Interpolate to get Coefficient of Power/Thrust for Motor RPM
    Cp = bottomCp + (RPM - bottomRPM) * (topCp - bottomCp) / (topRPM - bottomRPM)
    Ct = bottomCt + (RPM - bottomRPM) * (topCt - bottomCt) / (topRPM - bottomRPM)
    torque = bottomTorque + (RPM - bottomRPM) * (topTorque - bottomTorque) / (topRPM - bottomRPM)
    
    # Prop Power/Thrust Calc
    pProp = Density * RPS**3 * propDia**5 * Cp         # Power Prop requires in kg * m^2 / s^3 (W)
    ThrustProp = Density * RPS**2 * propDia**4 * Ct    # Thrust Prop generates in kg*m/2  (N)
    
    if (Output == 1):
        print('Export final parameters here')
        
    return pMotorIn, pMotorOut, pProp, vTerm, motorAmp, vBat, batAmp, ThrustProp, vEMF, RPM, torque

In [46]:
""" Final Calculations """

InitMotorGuess = nLCurr + 0.01    # Make First Motor Amp Draw Guess
InitBatGuess = nLCurr + 0.01      # No Load Amp Draw for Motor

# Dataframe for Storing Final Results
resultProp = pd.DataFrame(columns = [ "Battery Current (A)", "RPM (rev/min)", "Thrust (N)", "Power (W)", "Efficiency", "Torque (in-lbf)"])
resultProp.columns.name = 'Velocity (m/s)'

powerTolerance = 0.01 # Tolerance of power to Motor to Prop

velocityPosition = 0 # VelocityArray Position
velocity = velocityArray[velocityPosition] # Current Velocity in m/s

pMotorIn, pMotorOut, pProp, vTerm, motorAmp, vBat, batAmp, ThrustProp, vEMF, RPM, Torque = RPMfinder(InitMotorGuess, InitBatGuess, 0, velocity)

while (ThrustProp > 0):
    
    if (velocity != 0):
        velocityArray.append(velocity)
        
    timeRun = 0   # Counts how many times Loop iterates
    
    pMotorIn, pMotorOut, pProp, vTerm, motorAmp, vBat, batAmp, ThrustProp, vEMF, RPM, Torque = RPMfinder(InitMotorGuess, InitBatGuess, 0, velocity)
    
    
    while (abs(pProp - pMotorOut) > powerTolerance):
        pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBat,batAmp,ThrustProp,vEMF,RPM,Torque = RPMfinder(InitMotorGuess,InitBatGuess,0,velocity)

        print('Motor Power: ' + str(pMotorOut) + ' W')
        print('Thrust Prop: ' + str(ThrustProp) + ' N')
        print('Prop Power: ' + str(pProp) + ' W')
        print('pMotorIn: ' + str(pMotorIn) + ' W')
        print('prop calc done\n')
        
        if (pMotorOut < 0):
            raise Exception("Motor Power Out is negative, something is wrong")
        
        pESCOut = pMotorIn
        vESCin = (cells * batVolt)- (cells * batRes * batAmp)
        vESCout = (cells * batVolt) - (cells * batRes + mRes) * batAmp
        InitMotorGuess = pProp / vEMF + nLCurr
        InitBatGuess = pESCOut / vESCout
        
        timeRun = timeRun + 1
        print("T" + str(timeRun))
    
    # Runs one More Time (Could Keep previous motorAmp/BatAmp values, they are close to new)
    print("EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE")
    pMotorIn,pMotorOut,pProp,vTerm,motorAmp,vBat,batAmp,ThrustProp,vEMF,RPM,Torque = RPMfinder(InitMotorGuess,InitBatGuess,0,velocity)
    
    efficiency = pMotorOut / pMotorIn
    
    resultProp.loc[velocityArray[velocityPosition]] = [motorAmp, RPM, ThrustProp, pProp, efficiency, Torque]
    
    velocityPosition += 1
    velocity += velocityStep
    print("Max Velocity: " + str(velocityPosition) + " mph")
    

thrustLB = resultProp.loc[:, 'Thrust (N)'] * 0.224809
resultProp['Thrust (lbf)'] = thrustLB

# Removing Last Row b/c FN is 0 so results may be Invalid
resultProp.drop(resultProp.tail(1).index, inplace = True)
resultProp
resultProp.to_csv('TwinMotor.csv')
print("done")

Motor Power: 0.22138782000000023 W
Thrust Prop: 90.66564490023269 N
Prop Power: 2564.267106955708 W
pMotorIn: 0.2216443200000002 W
prop calc done

T1
Motor Power: 2367.141942231369 W
Thrust Prop: 77.26180200050226 N
Prop Power: 2169.1721557685933 W
pMotorIn: 2571.3336873649105 W
prop calc done

T2
Motor Power: 1928.9028779336888 W
Thrust Prop: 60.63610421468656 N
Prop Power: 1493.6276399039816 W
pMotorIn: 2100.5950791335104 W
prop calc done

T3
Motor Power: 1527.2314662898314 W
Thrust Prop: 63.485540469816364 N
Prop Power: 1604.348100130075 W
pMotorIn: 1630.6516030208334 W
prop calc done

T4
Motor Power: 1645.1958562522834 W
Thrust Prop: 66.86939666859462 N
Prop Power: 1739.550936191511 W
pMotorIn: 1759.2127866210283 W
prop calc done

T5
Motor Power: 1727.2163415540326 W
Thrust Prop: 65.89350782810585 N
Prop Power: 1700.1494999387323 W
pMotorIn: 1854.5529624863323 W
prop calc done

T6
Motor Power: 1692.0376828119709 W
Thrust Prop: 65.24573785288221 N
Prop Power: 1674.1784842980278 W
pM

Motor Power: 1293.6472839330047 W
Thrust Prop: 56.137307456954396 N
Prop Power: 1293.6576012482772 W
pMotorIn: 1359.4575671856805 W
prop calc done

T10
Motor Power: 1293.6583953975637 W
Thrust Prop: 56.137396011762355 N
Prop Power: 1293.66029793001 W
pMotorIn: 1359.4697150779816 W
prop calc done

T11
EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
Max Velocity: 9 mph
Motor Power: 1293.659519720075 W
Thrust Prop: 53.86108350791664 N
Prop Power: 1259.3356698816597 W
pMotorIn: 1359.4710305205024 W
prop calc done

T1
Motor Power: 1260.9922479098789 W
Thrust Prop: 54.046838915634794 N
Prop Power: 1264.987950167695 W
pMotorIn: 1323.4008869471024 W
prop calc done

T2
Motor Power: 1267.18513333813 W
Thrust Prop: 54.293076310465146 N
Prop Power: 1272.4894012632412 W
pMotorIn: 1329.9847471349854 W
prop calc done

T3
Motor Power: 1272.0461214596435 W
Thrust Prop: 54.24354656563417 N
Prop Power: 1270.9797085066557 W
pMotorIn: 1335.36573230422 W
prop calc done

T4
Motor Power: 1270.65504487304

Motor Power: 926.165937769335 W
Thrust Prop: 29.65906449974139 N
Prop Power: 926.3630350496313 W
pMotorIn: 957.820952211136 W
prop calc done

T7
Motor Power: 926.3352776402078 W
Thrust Prop: 29.655339250236633 N
Prop Power: 926.2387149096387 W
pMotorIn: 958.0035217428081 W
prop calc done

T8
Motor Power: 926.2321274062149 W
Thrust Prop: 29.65445510852245 N
Prop Power: 926.2092094794923 W
pMotorIn: 957.8938894381608 W
prop calc done

T9
Motor Power: 926.2142329781061 W
Thrust Prop: 29.65512935256564 N
Prop Power: 926.2317102252059 W
pMotorIn: 957.8744566600895 W
prop calc done

T10
Motor Power: 926.2323164305391 W
Thrust Prop: 29.65521071534574 N
Prop Power: 926.2344254548458 W
pMotorIn: 957.8937132508879 W
prop calc done

T11
EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
Max Velocity: 18 mph
Motor Power: 926.2335683573738 W
Thrust Prop: 26.24970671987706 N
Prop Power: 850.1810069936382 W
pMotorIn: 957.8951067447196 W
prop calc done

T1
Motor Power: 852.4804711411508 W
Thrust Pro

Motor Power: 271.92149284549953 W
Thrust Prop: 3.5901178291998113 N
Prop Power: 271.3206339267622 W
pMotorIn: 274.5896080278337 W
prop calc done

T4
Motor Power: 271.26979267593777 W
Thrust Prop: 3.578262707902263 N
Prop Power: 270.8769281614126 W
pMotorIn: 273.92683310173294 W
prop calc done

T5
Motor Power: 270.88716739148043 W
Thrust Prop: 3.580653091396864 N
Prop Power: 270.9663874141128 W
pMotorIn: 273.53697776690314 W
prop calc done

T6
Motor Power: 270.97047011316374 W
Thrust Prop: 3.581606025079951 N
Prop Power: 271.00205152908944 W
pMotorIn: 273.6217375631445 W
prop calc done

T7
Motor Power: 271.00085808016513 W
Thrust Prop: 3.5813274918540525 N
Prop Power: 270.99162720226894 W
pMotorIn: 273.6527065018736 W
prop calc done

T8
EEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEEE
Max Velocity: 26 mph
Motor Power: 270.9913212835127 W
Thrust Prop: 1.7906280466289748 N
Prop Power: 213.10127638313227 W
pMotorIn: 273.64299988668733 W
prop calc done

T1
Motor Power: 213.4933823656521

In [ ]:
print("Done")